In [1]:
import pandas as pd
import numpy as np

from pandas.api.types import is_string_dtype

pd.options.display.max_columns = None

In [2]:
port_clientes = pd.read_csv("../datasets/portfolio_clientes.csv", sep=",")
port_comunicados = pd.read_csv("../datasets/portfolio_comunicados.csv", sep=",")
port_geral = pd.read_csv("../datasets/portfolio_geral.csv", sep=",", nrows=200000, skiprows=range(1, 8000000, 40))
port_tpv = pd.read_csv("../datasets/portfolio_tpv.csv", sep=",")

In [3]:
def tratar_dataset(dataset):

    for coluna in dataset.columns:
        if is_string_dtype(dataset[coluna]):
            dataset[coluna] = dataset[coluna].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            dataset[coluna] = dataset[coluna].str.upper()
            
    dataset.columns = map(str.upper, dataset.columns)
    
    return dataset


def converter_para_datetime(dataset, lst_colunas, formato):
    for coluna in lst_colunas:
        dataset[coluna] = pd.to_datetime(dataset[coluna], format=formato)
    
    
def agrupa_e_segmenta_porcentagem(dataframe, coluna_groupby, coluna_tipo):
    tipos = list(filter(lambda elem: not pd.isna(elem), dataframe[coluna_tipo].unique()))
    
    tipos_dict = {}
    for tipo in tipos:
        query = dataframe.query(f'{coluna_tipo} == "{tipo}"')
        contagem = query.groupby(coluna_groupby).size()
        porcentagem = round(contagem / contagem.sum() * 100, 2)

        tipo_dataframe = pd.DataFrame()
        tipo_dataframe['contagem'] = contagem
        tipo_dataframe['porcentagem'] = porcentagem

        tipos_dict[tipo] = tipo_dataframe.sort_values(by=['contagem'], ascending=False)

    return tipos_dict


In [4]:
lst_datasets = [port_clientes, port_comunicados, port_geral, port_tpv]

for dataset in lst_datasets:
    dataset = tratar_dataset(dataset)
    dataset.dropna(inplace=True)
    dataset.drop_duplicates(inplace=True)


In [5]:
converter_para_datetime(port_tpv, ["DT_TRANSACAO"], "%Y%m%d")
converter_para_datetime(port_geral, ["DT_REF_PORTFOLIO", "SAFRA", "DT_CONTRATO", "DT_DESEMBOLSO", "DT_VENCIMENTO", "DT_WO"], "%Y-%m-%d")
converter_para_datetime(port_comunicados, ["DT_REF_PORTFOLIO", "DATA_ACAO"], "%Y-%m-%d")

In [6]:
port_clientes.drop(port_clientes.index[port_clientes["ESTADO"] == "ND"], inplace = True)
port_clientes.drop(port_clientes.index[port_clientes["SUBSEGMENTO"] == "NONE"], inplace = True)

In [7]:
lst_nr_suspensos = set(port_geral.query("STATUS_CONTRATO == 'SUSPENDED'").NR_DOCUMENTO.tolist())
lst_nr_pagos = set(port_geral.query("STATUS_CONTRATO == 'SETTLED'").NR_DOCUMENTO.tolist())

In [8]:
intersection_set = set.intersection(set(lst_nr_pagos), set(lst_nr_suspensos))

In [9]:
lst_nr_pago_sem_suspenso = lst_nr_pagos - lst_nr_suspensos

# PESSOAS QUE QUITARAM A DIVIDA E NUNCA FORAM SUSPENSAS

In [10]:
lst_nr_suspenso_sem_pagar = lst_nr_suspensos - lst_nr_pagos

# PESSOAS QUE FORAM SUSPENSAS E NUNCA PAGARAM A DIVIDA

In [11]:
port_comunicados

,CONTRATO_ID,DT_REF_PORTFOLIO,DATA_ACAO,TIPO_ACAO,ACAO,STATUS
0,0D823DDFF1839EB938DCCC68586AE365,2021-01-16,2021-01-18,HSM,CAMPANHAOBSERVACAO,LIDO
1,C2F8A21671DBAEAA4157E03682C639E1,2021-05-22,2021-05-24,HSM,CAMPANHAOBSERVACAO,ENTREGUE
2,E19A222C233D46FF4984489C13AA1B98,2020-09-15,2020-09-15,HSM,CAMPANHAOBSERVACAO,NAO ENTREGUE
3,C24FC7630FE5B89928BA9478C2CDD9B2,2021-07-20,2021-07-20,HSM,CAMPANHAOBSERVACAO,NAO ENTREGUE
4,0ABE9C2F21E941F2BD3B086277E33EBC,2020-12-26,2020-12-28,HSM,CAMPANHAOBSERVACAO,ENTREGUE
...,...,...,...,...,...,...
403735,C7DC25F386144912B90500FFD45E71C5,2020-10-19,2020-10-19,HSM,CAMPANHABOLETOQUITADO,NAO ENTREGUE
403736,76C9917AF4FA0810BD2E5638523A1A30,2020-06-19,2020-06-19,HSM,CAMPANHABOLETOQUITADO,LIDO
403737,724407E5260F93C0E3EA385564BDC731,2021-06-28,2021-06-28,HSM,CAMPANHABOLETOQUITADO,NAO ENTREGUE
403738,42B5D6594F79EE300CE0966C894B04C5,2021-09-26,2021-09-28,HSM,CAMPANHABOLETOQUITADO,NAO ENTREGUE


In [12]:
port_comunicados.groupby(by=["CONTRATO_ID"]).size().quantile(0.9)

76.0

In [13]:
contratos_pagos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(lst_nr_pago_sem_suspenso)]


In [14]:
contratos_pagos

,CONTRATO_ID,DT_REF_PORTFOLIO,SAFRA,NR_DOCUMENTO,STATUS_CONTRATO,DT_CONTRATO,DT_DESEMBOLSO,DT_VENCIMENTO,DT_WO,PRAZO,VLR_DESEMBOLSADO,VLR_TARIFA,JUROS_MES,JUROS_DIARIO,PERC_RETENCAO,VLR_PGTO_REALIZADO,VLR_PGTO_ESPERADO,VLR_SALDO_DEVEDOR,VLR_SALDO_DEVEDOR_ESPERADO,DSP,DSPP,FLAG_TRANSACAO
0,1EB0E13F83F336146227CCC9EFB08C47,2022-03-01,2020-07-01,938513461B4A4FCD2BF99FFD8F12B9BE,SETTLED,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,0.08400,0.00,46.87,0.00,0.00,0,0,0
1,8B03675F597974CA96F06648E96AF6D4,2022-03-01,2020-08-01,4303BD71EE3FE80ABA4EFC8DBD6C7D70,SETTLED,2020-08-13,2020-08-17,2022-02-13,2023-02-13,30.78,17100.00,0.0,0.05388,0.00176,0.08076,0.00,45.57,0.00,0.00,0,0,0
7,C48198C89F94CA4FD075BF30B889B47D,2020-10-22,2020-04-01,77846FD227FE5270DBBFE752515D1AF5,ACTIVE,2020-04-15,2020-04-22,2021-07-15,2022-07-15,25.65,1710.00,0.0,0.04788,0.00157,0.05520,30.95,5.03,466.54,1127.39,0,0,0
8,05EDDBA83DC40815A1AA10B6D6259708,2020-10-22,2020-07-01,62290FE7C7682B1F829AE81B1FD08BAF,ACTIVE,2020-07-02,2020-07-06,2021-10-02,2022-10-02,25.65,10514.79,0.0,0.07188,0.00233,0.13200,130.59,34.91,7285.04,8756.04,0,0,0
9,5D441EC4BDEDAA91E530CBFD72AC3966,2022-03-24,2020-07-01,D41D3E7F476557383C0CE4392448F55D,SETTLED,2020-07-21,2020-07-23,2021-10-21,2022-10-21,25.65,23921.19,0.0,0.04788,0.00157,0.10800,0.00,69.75,0.00,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199983,74DC2E4DE6CC77D14BF1AC36529A3A96,2021-11-24,2020-09-01,4E9FB2D22D306C7F94A22B798DA62E82,SETTLED,2020-09-04,2020-09-09,2022-06-04,2023-06-04,35.91,6253.47,0.0,0.05988,0.00195,1.20000,0.00,15.82,0.00,1481.14,0,0,0
199985,3FA4F7070A43186DC41EA2205117D093,2021-11-24,2020-02-01,53DC1E99CB8D17993C09CC3C0F75FBC0,SETTLED,2020-02-18,2020-02-21,2020-11-18,2021-11-18,13.68,6840.00,68.4,0.08388,0.00271,0.25740,0.00,34.23,0.00,0.00,0,0,0
199994,1D7FE0015E269FC1BFFC78E80D8789C7,2021-02-28,2020-08-01,DED4D7A9D876EBF9DFDD656B5AC9AE88,ACTIVE,2020-08-11,2020-08-13,2021-12-11,2022-12-11,27.36,19911.24,0.0,0.05988,0.00195,0.13200,0.00,59.35,14181.97,13534.26,4,11,0
199998,226DEF9CE410C15DDDE927801581BEC0,2021-11-12,2020-08-01,1393488C35CB0BC0C67ED857402B6FF1,SETTLED,2020-08-12,2020-08-14,2021-12-12,2022-12-12,27.36,9442.62,0.0,0.09588,0.00308,1.20000,0.00,34.06,0.00,0.00,0,0,0


In [15]:
contratos_pagos_suspensos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(intersection_set)]

In [16]:
contratos_suspensos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(lst_nr_suspenso_sem_pagar)]

############################################################################


In [17]:
clientes_com_contratos_pago = contratos_pagos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [18]:
contratos_pagos_suspensos_merge = contratos_pagos_suspensos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [19]:
clientes_com_contratos_suspensos = contratos_suspensos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [20]:
len(contratos_pagos_suspensos_merge.NR_DOCUMENTO.unique()), len(contratos_pagos_suspensos_merge)

(458, 9718)

############################################################################

In [21]:
def agrupa_e_segmenta_porcentagem(dataframe, coluna_groupby, coluna_tipo):
    tipos = list(filter(lambda elem: not pd.isna(elem), dataframe[coluna_tipo].unique()))
    
    tipos_dict = {}
    for tipo in tipos:
        query = dataframe.query(f'{coluna_tipo} == "{tipo}"')
        contagem = query.groupby(coluna_groupby).size()
        porcentagem = round(contagem / contagem.sum() * 100, 2)

        tipo_dataframe = pd.DataFrame()
        tipo_dataframe['contagem'] = contagem
        tipo_dataframe['porcentagem'] = porcentagem

        tipos_dict[tipo] = tipo_dataframe.sort_values(by=['contagem'], ascending=False)

    return tipos_dict

## Bom pagador

In [22]:
def gera_df_porcentagem(dataframe, tipo):
    contagem = dataframe.drop_duplicates('NR_DOCUMENTO').groupby(tipo).size()
    porcentagem = round(contagem / contagem.sum() * 100,2)

    dataframe_resultado = pd.DataFrame()
    dataframe_resultado['contagem'] = contagem
    dataframe_resultado['porcentagem'] = porcentagem

    return dataframe_resultado.sort_values('contagem', ascending=False)

### Tipo empresa

In [23]:
bom_pagador_tipo_empresa = gera_df_porcentagem(clientes_com_contratos_pago, 'TIPO_EMPRESA')

### Segmento

In [24]:
bom_pagador_segmento = gera_df_porcentagem(clientes_com_contratos_pago, 'SEGMENTO')

### Status contrato

In [25]:
bom_pagador_status_contrato = gera_df_porcentagem(clientes_com_contratos_pago, 'STATUS_CONTRATO')

### Estado

In [26]:
bom_pagador_estado = gera_df_porcentagem(clientes_com_contratos_pago, 'ESTADO')

## Mau pagador

### Tipo empresa

In [27]:
mau_pagador_tipo_empresa = gera_df_porcentagem(clientes_com_contratos_suspensos, 'TIPO_EMPRESA')

### Segmento

In [28]:
mau_pagador_segmento = gera_df_porcentagem(clientes_com_contratos_suspensos, 'SEGMENTO')

### Status contrato

In [29]:
mau_pagador_status_contrato = gera_df_porcentagem(clientes_com_contratos_suspensos, 'STATUS_CONTRATO')

### Estado

In [30]:
mau_pagador_estado = gera_df_porcentagem(clientes_com_contratos_suspensos, 'ESTADO')

In [33]:
mau_pagador_estado

,contagem,porcentagem
ESTADO,,
SP,204,24.23
RJ,132,15.68
PR,69,8.19
BA,52,6.18
DF,38,4.51
MG,38,4.51
GO,36,4.28
SC,35,4.16
CE,34,4.04


Prazo/parcelas do contrato da linha de crédito, geralmente o prazo médio é de 8 meses.

https://www.bndes.gov.br/wps/portal/site/home/financiamento/guia/prazos-periodicidade-pagamento

In [31]:
lst_nr_pago_sem_suspenso = list(lst_nr_pago_sem_suspenso)

In [34]:
mau_pagador_estado.to_csv('mau_pagador_estado.csv')
mau_pagador_status_contrato.to_csv('mau_pagador_status_contrato.csv')
mau_pagador_segmento.to_csv('mau_pagador_segmento.csv')
mau_pagador_tipo_empresa.to_csv('mau_pagador_tipo_empresa.csv')
bom_pagador_estado.to_csv('bom_pagador_estado.csv')
bom_pagador_status_contrato.to_csv('bom_pagador_status_contrato.csv')
bom_pagador_segmento.to_csv('bom_pagador_segmento.csv')
bom_pagador_tipo_empresa.to_csv('bom_pagador_tipo_empresa.csv')

Estado atual do empréstimo, podendo ser "Desembolso confirmado (DisbursementConfirmed), Aceito (Accepted), Desembolso solicitado (DisbursementRequested), Ativo (Active) ou Quitado (Settled)".